In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, year, month
from pyspark.sql.functions import isnull

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%bash
# This is a Bash cell
wget -O - https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz | aws s3 cp - s3://emr-raw-data-dak/city_market_tracker.tsv000.gz

--2025-09-07 14:21:10--  https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz
Resolving redfin-public-data.s3.us-west-2.amazonaws.com (redfin-public-data.s3.us-west-2.amazonaws.com)... 52.92.180.146, 3.5.78.38, 3.5.84.190, ...
Connecting to redfin-public-data.s3.us-west-2.amazonaws.com (redfin-public-data.s3.us-west-2.amazonaws.com)|52.92.180.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 948653269 (905M) [binary/octet-stream]
Saving to: ‘STDOUT’

     0K .......... .......... .......... .......... ..........  0%  394K 39m9s
    50K .......... .......... .......... .......... ..........  0%  390K 39m21s
   100K .......... .......... .......... .......... ..........  0%  270M 26m15s
   150K .......... .......... .......... .......... ..........  0%  395K 29m27s
   200K .......... .......... .......... .......... ..........  0%  120M 23m35s
   250K .......... .......... .......... .......... ..........  0

In [3]:
Spark = SparkSession.builder.appName("CityMarketTracker").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
redfin_data = spark.read.csv("s3://emr-raw-data-dak/city_market_tracker.tsv000.gz", sep="\t", header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
redfin_data.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------+---------------+-----------+--------------+--------+----------------------+----------------+------------+----------+----------+--------------------+----------------+-----------------+---------------------+---------------------+-----------------+---------------------+---------------------+-------------+---------------+---------------+----------------+--------------------+--------------------+----------+--------------+--------------+-------------+-----------------+-----------------+------------+----------------+----------------+---------+-------------+-------------+----------------+--------------------+--------------------+----------+--------------+--------------+----------------+--------------------+--------------------+---------------+-------------------+-------------------+------------+---------------+---------------+-----------------------+---------------------------+---------------------------+-------------------+------------------------------+--------------

In [7]:
#check the schema
redfin_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- PERIOD_BEGIN: date (nullable = true)
 |-- PERIOD_END: date (nullable = true)
 |-- PERIOD_DURATION: integer (nullable = true)
 |-- REGION_TYPE: string (nullable = true)
 |-- REGION_TYPE_ID: integer (nullable = true)
 |-- TABLE_ID: integer (nullable = true)
 |-- IS_SEASONALLY_ADJUSTED: boolean (nullable = true)
 |-- REGION: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- STATE_CODE: string (nullable = true)
 |-- PROPERTY_TYPE: string (nullable = true)
 |-- PROPERTY_TYPE_ID: integer (nullable = true)
 |-- MEDIAN_SALE_PRICE: string (nullable = true)
 |-- MEDIAN_SALE_PRICE_MOM: string (nullable = true)
 |-- MEDIAN_SALE_PRICE_YOY: string (nullable = true)
 |-- MEDIAN_LIST_PRICE: string (nullable = true)
 |-- MEDIAN_LIST_PRICE_MOM: string (nullable = true)
 |-- MEDIAN_LIST_PRICE_YOY: string (nullable = true)
 |-- MEDIAN_PPSF: string (nullable = true)
 |-- MEDIAN_PPSF_MOM: string (nullable = true)
 |-- MEDIAN_PPSF_YOY: string (nu

In [8]:
#print column names
redfin_data.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['PERIOD_BEGIN', 'PERIOD_END', 'PERIOD_DURATION', 'REGION_TYPE', 'REGION_TYPE_ID', 'TABLE_ID', 'IS_SEASONALLY_ADJUSTED', 'REGION', 'CITY', 'STATE', 'STATE_CODE', 'PROPERTY_TYPE', 'PROPERTY_TYPE_ID', 'MEDIAN_SALE_PRICE', 'MEDIAN_SALE_PRICE_MOM', 'MEDIAN_SALE_PRICE_YOY', 'MEDIAN_LIST_PRICE', 'MEDIAN_LIST_PRICE_MOM', 'MEDIAN_LIST_PRICE_YOY', 'MEDIAN_PPSF', 'MEDIAN_PPSF_MOM', 'MEDIAN_PPSF_YOY', 'MEDIAN_LIST_PPSF', 'MEDIAN_LIST_PPSF_MOM', 'MEDIAN_LIST_PPSF_YOY', 'HOMES_SOLD', 'HOMES_SOLD_MOM', 'HOMES_SOLD_YOY', 'PENDING_SALES', 'PENDING_SALES_MOM', 'PENDING_SALES_YOY', 'NEW_LISTINGS', 'NEW_LISTINGS_MOM', 'NEW_LISTINGS_YOY', 'INVENTORY', 'INVENTORY_MOM', 'INVENTORY_YOY', 'MONTHS_OF_SUPPLY', 'MONTHS_OF_SUPPLY_MOM', 'MONTHS_OF_SUPPLY_YOY', 'MEDIAN_DOM', 'MEDIAN_DOM_MOM', 'MEDIAN_DOM_YOY', 'AVG_SALE_TO_LIST', 'AVG_SALE_TO_LIST_MOM', 'AVG_SALE_TO_LIST_YOY', 'SOLD_ABOVE_LIST', 'SOLD_ABOVE_LIST_MOM', 'SOLD_ABOVE_LIST_YOY', 'PRICE_DROPS', 'PRICE_DROPS_MOM', 'PRICE_DROPS_YOY', 'OFF_MARKET_IN_TWO_WEE

In [9]:
#check the count of rows
redfin_data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5978674

In [10]:
#check the count of distinct rows
redfin_data.distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5978674

In [11]:
#selecting specific columns for analysis
df_redfin = redfin_data.select(['period_end','period_duration', 'city', 'state', 'property_type',
    'median_sale_price', 'median_ppsf', 'homes_sold', 'inventory', 'months_of_supply', 'median_dom', 'sold_above_list', 'last_updated'])
df_redfin.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+------------+----------+--------------------+-----------------+-------------+----------+---------+----------------+----------+---------------+--------------------+
|period_end|period_duration|        city|     state|       property_type|median_sale_price|  median_ppsf|homes_sold|inventory|months_of_supply|median_dom|sold_above_list|        last_updated|
+----------+---------------+------------+----------+--------------------+-----------------+-------------+----------+---------+----------------+----------+---------------+--------------------+
|2012-07-31|             30|     Guthrie|  Oklahoma|Single Family Res...|            91500|  57.32442348|        16|       NA|             5.1|        NA|         0.1875|2025-08-01 14:37:...|
|2024-11-30|             30|    Milliken|  Colorado|           Townhouse|           390000|167.741935484|         1|        9|               9|        13|              0|2025-08-01 14:37:...|
|2017-10-31|             30|Palm Springs

In [14]:
#count the number of null values in each selected column for analysis
null_counts = df_redfin.select([count(when(isnull(c), c)).alias(c) for c in df_redfin.columns])
null_counts.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+----+-----+-------------+-----------------+-----------+----------+---------+----------------+----------+---------------+------------+
|period_end|period_duration|city|state|property_type|median_sale_price|median_ppsf|homes_sold|inventory|months_of_supply|median_dom|sold_above_list|last_updated|
+----------+---------------+----+-----+-------------+-----------------+-----------+----------+---------+----------------+----------+---------------+------------+
|         0|              0|   0|    0|            0|                0|          0|         0|        0|               0|         0|              0|           0|
+----------+---------------+----+-----+-------------+-----------------+-----------+----------+---------+----------------+----------+---------------+------------+

In [16]:
#count number of null values in each column of the dataset
null_counts = redfin_data.select([count(when(col(c).isNull(), c)).alias(c) for c in df_redfin.columns])
null_counts.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+----+-----+-------------+-----------------+-----------+----------+---------+----------------+----------+---------------+------------+
|period_end|period_duration|city|state|property_type|median_sale_price|median_ppsf|homes_sold|inventory|months_of_supply|median_dom|sold_above_list|last_updated|
+----------+---------------+----+-----+-------------+-----------------+-----------+----------+---------+----------------+----------+---------------+------------+
|         0|              0|   0|    0|            0|                0|          0|         0|        0|               0|         0|              0|           0|
+----------+---------------+----+-----+-------------+-----------------+-----------+----------+---------+----------------+----------+---------------+------------+

In [18]:
#display the result of the null counts
for i, col_name in enumerate(df_redfin.columns):
    print(f"Number of null values in column {col_name}: {null_counts.collect()[0][i]}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of null values in column period_end: 0
Number of null values in column period_duration: 0
Number of null values in column city: 0
Number of null values in column state: 0
Number of null values in column property_type: 0
Number of null values in column median_sale_price: 0
Number of null values in column median_ppsf: 0
Number of null values in column homes_sold: 0
Number of null values in column inventory: 0
Number of null values in column months_of_supply: 0
Number of null values in column median_dom: 0
Number of null values in column sold_above_list: 0
Number of null values in column last_updated: 0

In [19]:
# Display the results
for i, col_name in enumerate(df_redfin.columns):
    print(f"{col_name}: {null_counts[i]} null values")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

period_end: Column<'period_end'> null values
period_duration: Column<'period_duration'> null values
city: Column<'city'> null values
state: Column<'state'> null values
property_type: Column<'property_type'> null values
median_sale_price: Column<'median_sale_price'> null values
median_ppsf: Column<'median_ppsf'> null values
homes_sold: Column<'homes_sold'> null values
inventory: Column<'inventory'> null values
months_of_supply: Column<'months_of_supply'> null values
median_dom: Column<'median_dom'> null values
sold_above_list: Column<'sold_above_list'> null values
last_updated: Column<'last_updated'> null values

In [21]:
#Extract the year from the period_end column
df_redfin = df_redfin.withColumn("year", year(col("period_end")))

#Extract the month from the period_end column
df_redfin = df_redfin.withColumn("month", month(col("period_end")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
#drop the period_end column and last_updated
df_redfin = df_redfin.drop("period_end", "last_updated")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
df_redfin.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+------------+----------+--------------------+-----------------+-------------+----------+---------+----------------+----------+---------------+----+-----+
|period_duration|        city|     state|       property_type|median_sale_price|  median_ppsf|homes_sold|inventory|months_of_supply|median_dom|sold_above_list|year|month|
+---------------+------------+----------+--------------------+-----------------+-------------+----------+---------+----------------+----------+---------------+----+-----+
|             30|     Guthrie|  Oklahoma|Single Family Res...|            91500|  57.32442348|        16|       NA|             5.1|        NA|         0.1875|2012|    7|
|             30|    Milliken|  Colorado|           Townhouse|           390000|167.741935484|         1|        9|               9|        13|              0|2024|   11|
|             30|Palm Springs|California|     All Residential|           352000|224.632068164|       176|      613|             3.5|        61|  

In [24]:
#let then map the month column to the corresponding month name
df_redfin = df_redfin.withColumn("month", when(col("month") == 1, "January")
                                   .when(col("month") == 2, "February")
                                   .when(col("month") == 3, "March")
                                   .when(col("month") == 4, "April")
                                   .when(col("month") == 5, "May")
                                   .when(col("month") == 6, "June")
                                   .when(col("month") == 7, "July")
                                   .when(col("month") == 8, "August")
                                   .when(col("month") == 9, "September")
                                   .when(col("month") == 10, "October")
                                   .when(col("month") == 11, "November")
                                   .when(col("month") == 12, "December")
                                   .otherwise("Invalid Month"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
df_redfin.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+------------+----------+--------------------+-----------------+-------------+----------+---------+----------------+----------+---------------+----+--------+
|period_duration|        city|     state|       property_type|median_sale_price|  median_ppsf|homes_sold|inventory|months_of_supply|median_dom|sold_above_list|year|   month|
+---------------+------------+----------+--------------------+-----------------+-------------+----------+---------+----------------+----------+---------------+----+--------+
|             30|     Guthrie|  Oklahoma|Single Family Res...|            91500|  57.32442348|        16|       NA|             5.1|        NA|         0.1875|2012|    July|
|             30|    Milliken|  Colorado|           Townhouse|           390000|167.741935484|         1|        9|               9|        13|              0|2024|November|
|             30|Palm Springs|California|     All Residential|           352000|224.632068164|       176|      613|             3.